In [11]:
%load_ext autoreload
%autoreload 2

import DataValidator.DataValidator as dv_mod
import pandas as pd

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [51]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', 30)
pd.set_option('display.float_format', '{:,.2f}'.format)

In [2]:
dv = dv_mod.DataValidator()

Loading cached data…


In [3]:
QUANTITATIVE_COLS = ["net_stake", "ggr", "decimalodds"]
QUALITATIVE_COLS = ["state", "sportname", "bet_type", "result", "legresult"]
DATE_COLS = ["event_start", "placed_date", "settled_date"]
ID_COLS = ["playerid", "wagerid"]

In [5]:
print("count null values")
for col in dv.project_df.columns:
    print(col, dv.count_na(col))

count null values
state 0
playerid 0
wagerid 0
event_start 453
placed_date 0
settled_date 0
sportname 0
bet_type 0
result 0
net_stake 0
ggr 0
legresult 0
decimalodds 2472


In [8]:
print("distinct values")
for col in QUALITATIVE_COLS:
    print(col)
    print("  ", dv.get_distinct(col))

distinct values
state
   ['State1' 'State2' 'State3']
sportname
   ['champions league' 'college basketball' 'college football' 'mlb' 'nba'
 'nfl' 'nhl']
bet_type
   ['parlay' 'straight']
result
   ['lost' 'won']
legresult
   ['lost' 'open' 'unknown' 'void' 'won']


In [17]:
print("Summary Stats - Quantitative Cols")
for col in QUANTITATIVE_COLS:
    print()
    print(dv.numeric_summary_stats(col))

Summary Stats - Quantitative Cols

                 net_stake
total_count      4,174,500
missing_count            0
missing_percent          0
unique_count        31,701
mean                    24
std                     82
min                      4
25%                      5
50% (median)            10
75%                     20
max                 19,672
outlier_count       33,335
skew                    40
kurtosis             4,039

                      ggr
total_count     4,174,500
missing_count           0
missing_percent         0
unique_count       96,979
mean                    2
std                    85
min               -21,000
25%                     0
50% (median)            6
75%                    12
max                19,672
outlier_count      42,429
skew                   -8
kurtosis            3,713

                 decimalodds
total_count        4,174,500
missing_count          2,472
missing_percent            0
unique_count          18,694
mean                   

In [15]:
print("Summary Stats - Date Cols")
for col in DATE_COLS:
    print()
    print(dv.date_summary_stats(col))

Summary Stats - Date Cols

                               event_start
total_count                        4174500
missing_count                          453
missing_percent                          0
earliest_date    2021-03-20 23:00:00+00:00
latest_date      2023-02-12 23:30:00+00:00
date_range_days                        694

                         placed_date
total_count                  4174500
missing_count                      0
missing_percent                    0
earliest_date    2021-03-28 00:00:00
latest_date      2022-03-28 00:00:00
date_range_days                  365

                        settled_date
total_count                  4174500
missing_count                      0
missing_percent                    0
earliest_date    2021-03-28 00:00:00
latest_date      2022-03-29 00:00:00
date_range_days                  366


In [16]:
print("Summary Stats - ID Cols")
for col in ID_COLS:
    print()
    print(dv.id_summary_stats(col))

Summary Stats - ID Cols

                playerid
data_type         object
total_count      4174500
unique_count       36387
missing_count          0
missing_percent        0

                 wagerid
data_type         object
total_count      4174500
unique_count     2402300
missing_count          0
missing_percent        0
